In [1]:
from logger import Logger
from preparator import Preparator 
from solver import Solver 
import pandas as pd 
from tqdm import tqdm 
from datetime import datetime

import pennylane as qml 
from pennylane import numpy as np
testdev = qml.device('default.qubit', wires = 1)

In [2]:
size = 10
matrix_number = 50
source_name = 'Traininig_size_{size}.csv'

template = pd.read_csv('data/template.csv')

In [3]:
# Prepare train datasets
log = Logger (log_template = template, log_path = f'data/train/{source_name.format(size = size)}')
exists = log.create_logfile()

In [4]:
log_file, id = log._load_logfile()

prep = Preparator()

for i in tqdm(range(matrix_number), desc = f"Generating random matrices with size {size}"):
    
    id += 1
    matrix = prep.rankPreparator(size = size, rank = size)
    current_datetime = datetime.now()
    data = { 
        'Id (int)' : id, 
        'Valid (int)' : 1, 
        'Updated (str)' : current_datetime.strftime("%Y-%m-%d %H:%M:%S"),
        'Matrix (mat)' : matrix,
        'Size (int)' : size,
        'Type (str)' : 'Random',
        'Rank (int)' : size,
        'Density (flt)' : -1.0,
        'Stiffness (flt)' : -1.0,
        'Other (flt)' : -1.0,
        'Test_solver (str)' : '-',
        'Test (flt)' : -1.0,
        'Test_state (vec)' : [],
        'Test_iterations (int)' : -1,
        'Test_time (flt)' : -1.0,
        'Train_solver (str)' : '-',
        'Preprocessing (str)' : '-',
        'Postprocessing (str)' : '-',
        'Train (flt)' : -1.0,
        'Train_state (vec)' : [],
        'Train_iterations_1 (int)' : -1,
        'Train_iterations_2 (int)' : -1,
        'Train_iterations_3 (int)' : -1,
        'Train_shots (int)': -1, 
        'Train_cost (flt)' : -1.0,
        'Train_time (flt)' : -1.0, 
        'Flags (str)' : ''
    }
    
    log.log(data)



Generating random matrices with size 10:  36%|███▌      | 18/50 [00:00<00:00, 55.83it/s]

Generating random matrices with size 10: 100%|██████████| 50/50 [00:01<00:00, 30.79it/s]


In [5]:

def test (ids : np.ndarray, solver : str = 'bruteforce', log_data : bool = False, progressbar : bool = False ):

    ''' 
    ids : np.ndarray 
        a list of ids to compute 
    solver : str 
        solver to compute with 
    ''' 
    
    tester = Solver(dev = testdev, ansatz = '')

    if (progressbar):
        id_range = tqdm(ids, desc = f"Testing matrices with ids {min(ids)} <--> {max(ids)}") 
    else: 
        id_range = ids
    processed_data = []
    for id in id_range:

        data = log.read(id)

        
        def test_solver(Q : np.ndarray, solver : str = 'bruteforce'): 
            ''' 
            tester wrapped
            '''

            match solver: 
                case 'bruteforce':
                    J, h = prep.isingForm(Q)
                    return tester.checkIsing(J, h)
                case 'GW':
                    pass
                case 'Annealing': 
                    pass
            
        Q = data['Matrix (mat)']

        tested_data = data.copy()
        
        test, test_state, test_time = test_solver(Q)
        current_datetime = datetime.now()

        tested_data['Test (flt)'] = test 
        tested_data['Test_state (vec)'] = test_state[0].numpy()
        tested_data['Test_time (flt)'] = test_time
        tested_data['Test_solver (flt)'] = solver
        tested_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")

        processed_data.append(tested_data)

        if (log_data): 
            log.log(tested_data, byid = True, Id = id)
    return processed_data

In [6]:

ids = range (1,51)
number_of_solvers = 10
poolsize = len(ids)//number_of_solvers

In [7]:

from multiprocessing import Pool
from functools import partial

if (__name__ == '__main__') and (len(ids) % poolsize == 0): 
    
    solvers = [] 

    ids = np.array(ids) 
    index = 0
    pool_ids = ids.reshape(len(ids) // poolsize, poolsize)
    display(pool_ids)

    args = []
    for i in range (len(pool_ids)):
        if (size > 11):
            args.append((pool_ids[i], 'GW'))
        else: 
            args.append((pool_ids[i], 'bruteforce'))
    #print(args)
    with Pool(number_of_solvers) as pool:
        processed_data = pool.starmap(test, args)


for i in range(number_of_solvers):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])

# If the error occured you still can try to get processed_data 


tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10],
        [11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20],
        [21, 22, 23, 24, 25],
        [26, 27, 28, 29, 30],
        [31, 32, 33, 34, 35],
        [36, 37, 38, 39, 40],
        [41, 42, 43, 44, 45],
        [46, 47, 48, 49, 50]], requires_grad=True)

In [8]:
display(processed_data)

[[{'Id (int)': 1,
   'Valid (int)': 1,
   'Updated (str)': '2025-08-20 17:43:29',
   'Matrix (mat)': array([[ 0.1759589 , -0.05986243,  0.15619587, -0.20446538, -0.17612252,
            0.00749069, -0.03662951,  0.28730247,  0.05681903,  0.10323369],
          [-0.05986243, -0.02150627, -0.12476013, -0.09734574, -0.1371941 ,
           -0.05410635, -0.0918655 , -0.04404508, -0.033297  , -0.0695647 ],
          [ 0.15619587, -0.12476013,  0.01812002, -0.46689733, -0.49337011,
           -0.08267553, -0.21278288,  0.34821331,  0.02542184,  0.03228501],
          [-0.20446538, -0.09734574, -0.46689733, -0.42924772, -0.58238771,
           -0.21552422, -0.37274405, -0.11973918, -0.12147453, -0.25685302],
          [-0.17612252, -0.1371941 , -0.49337011, -0.58238771, -0.75348159,
           -0.25212654, -0.45466187, -0.0344236 , -0.12213057, -0.26551693],
          [ 0.00749069, -0.05410635, -0.08267553, -0.21552422, -0.25212654,
           -0.06321604, -0.12954467,  0.0783582 , -0.0139262 

In [9]:
display(len(processed_data))

10

In [10]:

def train (ids : np.ndarray, trainer : Solver, depth : int = 1, log_data = False, progressbar : bool = False): 
    ''' 
    ids : np.ndarray 
        array of ids to train
    solver : str 
        solver with which will be trained 
    preprocessing : str 
    postprocessing : str 
    '''

    #print(f'training ids {ids}')
    if (progressbar):
        id_range = tqdm.tqdm(ids, desc = f"Training matrices with ids {min(ids)} <--> {max(ids)}")
    else:
        id_range = ids
    for id in id_range:
        
        data = log.read(id)

        
        def train_solver(Q : np.ndarray): 
            ''' 
            trainer wrapped
            '''
            return trainer.solve(Q = Q, depth = depth, sol = data['Test (flt)'], stepsize = 0.1, log = False, gate = 'Z', barencheck = False)
        
        Q = data['Matrix (mat)']

        trained_data = data.copy()
        
        train, train_bitstring, train_state, train_iterations, train_time = train_solver(Q)

        trained_data['Train (flt)'] = train
        trained_data['Train_state (vec)'] = train_bitstring
        for i in range(1, 4): trained_data[f'Train_iterations_{i} (int)'] = train_iterations[i - 1]
        trained_data['Train_time (flt)'] = train_time
        
        current_datetime = datetime.now()
        trained_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")

        
        if (log_data):
            log.log(trained_data, byid = True, Id = id)
        else:
            return trained_data


In [11]:

stepsizes = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 1.0, 1.5, 2.0]
poolcount = len(stepsizes)

solvers = [] 

for i in range (poolcount):
    solvers.append(Solver())

TypeError: Solver.__init__() missing 1 required positional argument: 'dev'